<h1> BERT: Pre-training of Deep Bidirectional Transformers for
Language Understanding </h1>

Understanding the "Attention is All You Need" paper is crucial to grasp the underpinnings of the BERT model. It provides the necessary background on the architecture and mechanism that BERT is based on, and also gives insights into the problems that BERT was designed to solve.

The paper "Attention is All You Need" by Vaswani et al., (2017) [[1]](#ref1) introduces the concept of the Transformer model, which is a type of model that uses a mechanism called attention to improve the understanding of sequence data, and especially natural language data. The key concept introduced in this paper is the self-attention mechanism, which allows the model to consider different words in the input sentence when encoding a particular word. Refer to the in-depth and illustrated Pytorch implementation of the paper for detailed study <a href="https://github.com/elnaramammadova/Attention_is_all_you_need/tree/master">here</a>. 

This concept of attention and specifically the Transformer model forms the backbone of the BERT (Bidirectional Encoder Representations from Transformers) model, which was proposed in a subsequent paper "BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding" by Devlin et al. (2018) [[2]](#ref2).

BERT utilizes the Transformer model in a unique way: instead of using both the encoder and decoder parts of the original Transformer model, BERT only uses the encoder part and is trained in a bidirectional manner, which means it learns to understand the context of a word based on all of its surroundings (left and right of the word). The attention mechanism allows BERT to focus on relevant parts of the input sentence when encoding each word, hence capturing the context more effectively.

BERT can be used on a wide variety of language tasks:

- Sentiment Analysis (i.e., reviews)
- Question answering (i.e., chatbots)
- Text prediction (i.e., writing an email)
- Text generation
- Summarization
- Polysemy resolution

BERT's remarkable performance owes a lot to its extensive training on a colossal dataset of 3.3 billion words. The learning material comprised Wikipedia, with around 2.5 billion words, and Google's BooksCorpus, contributing another 800 million words. This vast informational treasury enabled BERT to gain profound insights, not only into the intricacies of English language but also into the diverse facets of our world.

<h2>Import Libraries</h2>

In [ ]:
from IPython.display import Image 
import pandas as pd
from datasets import load_dataset # HuggingFace open-source library of generic dataset class

import torch # PyTorch open-source library of generic deep learning classes
# from torch import nn # neural network module

<h2>Introduction</h2>

<h3>Pre-Trained Language Models</h3>

Using pre-training methods for language models has proven to be really beneficial in improving many tasks related to processing natural language - how computers understand human language. Pre-trained word embeddings are an integral part of modern NLP systems, offering significant improvements over embeddings learned from scratch (Turian et al., 2010 [[3]](#ref3)).This has been proven by several studies conducted by various researchers.

There are different kinds of tasks that have seen improvements because of these pre-training methods. Some tasks, like natural language inference and paraphrasing, involve analyzing entire sentences to predict how they relate to each other. For example, determining whether a sentence is a restatement of another one or if one sentence logically follows from another.

On the other hand, there are tasks like named entity recognition and question answering, which need a more detailed analysis at the level of individual words or tokens. For these tasks, the models need to provide very specific outputs for each token. For instance, in named entity recognition, the model might need to identify that a token is a person's name or a location. In question answering, it might need to identify the specific parts of the text that answer a given question. Pre-training methods have shown to improve performance on these tasks as well.

<h3>Context-Independent to Context-Sensitive</h3>

Models like word2vec [[4]](#ref4) and GloVe [[5]](#ref5) assign the same pre-trained vector to a word, regardless of its context. This is like labelling a "bat" as a baseball tool, even when it's used in a sentence about animals. Given the complexity and nuances in language, this one-size-fits-all approach can be limiting.

For example, the word "apple" would have the same representation in "I ate an apple" and "Apple released a new iPhone". It's clear that "apple" means different things in these contexts.

To address this, context-sensitive models were developed. In these models, the representation of a word depends on its surrounding context. It's like understanding that "bat" in "The bat flew away" refers to an animal, not sports equipment.

Context-sensitive models include TagLM, CoVe, and ELMo. They allow the model to generate different representations for "apple" in our earlier examples, better capturing the intended meaning.

- <h4>ELMo LSTM Model</h4>

Let's look at ELMo. ELMo adjusts the representation of each word based on the context it's used in. To do this, it uses a pre-trained model, specifically a bidirectional LSTM, to process the whole sentence. Then, it takes the output from all the intermediate layers of this model and combines them to generate a unique representation for each word [Figure 1](#ref1).

Next, this unique, context-sensitive representation is added to the original model (like GloVe) used in the downstream task as additional features. It's like giving the model extra information about each word based on the context. While doing this, the weights (or parameters) in the pre-trained LSTM model remain constant.

This technique of adding ELMo to different task-specific models led to improvements in performance across six different language processing tasks like sentiment analysis, question answering, and more. It's like giving your model a context-sensitive superpower!

- <h4>OpenAI Transformer Model - GPT</h4>

Unlike ELmo, built on a Transformer decoder, GPT pretrains a language model that will be used to represent text sequences. Once GPT is trained and ready, it's applied to a specific downstream task. The knowledge it gained from its training is then combined with an additional layer to make predictions specific to the task at hand. Unlike ELMo, which keeps its pretrained model parameters fixed, GPT fine-tunes all its parameters during the learning process for the task, essentially allowing it to adjust and learn based on the new information it encounters. However, GPT only looks forward (left-to-right); it only looks at the words before the current word to predict its meaning [Figure 2](#fig2).

For example, consider the word "bank" in the sentence "I went to the bank to deposit cash." Here, "bank" means a place where you deposit money. But in the sentence "I went to the bank to sit down," "bank" means the edge of a river. Since GPT only reads the sentence up to the word "bank" and doesn't consider what comes after, it won't know the correct context of "bank" because it doesn't see the words "deposit cash" or "sit down." That's why GPT might interpret "bank" the same way in both sentences, even though it has different meanings.

- <h4>BERT</h4>

BERT, is a model that uses a pre-trained Transformer encoder structure, learns to understand the context of words in both directions - left and right. When we use BERT for specific tasks, we feed its representations into an added layer that's specially designed for that task. Like GPT, BERT adjusts all of its pre-trained parameters during this learning phase. At the same time, it's training the new output layer from scratch. In essence, BERT combines the best of both ELMo and GPT: it understands context from both directions like ELMo, and it fine-tunes all its parameters like GPT, resulting in a powerful model for natural language processing tasks [Figure 3](#fig3).

<h3>Bidirectional Representations</h3>

Existing strategies for applying pre-trained language representations to downstream tasks has a design that reads text from left to right, just like how we usually read a book or an article. In this design, each word or 'token' in the text can only pay attention to the words that came before it when the model is figuring out the meaning of a sentence or a phrase.

However, this left-to-right only approach has its limitations. For tasks where understanding the whole sentence is important, only being able to look back at previous words is not ideal. The meaning of a sentence often depends on all of its words, not just the ones before a certain point.

This limitation becomes even more problematic for tasks like question answering that require understanding each word or 'token' in relation to the whole text. In such tasks, understanding the context from both directions, i.e., from words before and after a certain point, is really important. 

BERT alleviates these constrains by using two specific methods. BERT's training involves simultaneous learning from both Masked Language Modelling (MLM) and Next Sentence Prediction (NSP), each contributing 50% to the training regimen.


In [ ]:
Image(filename='images/ELMo.png')

<a id="fig1">Figure 1.</a> ELMo comes up with the contextualized embedding through grouping together the hidden states (and initial embedding) in a certain way (concatenation followed by weighted summation). (source: <a href="http://jalammar.github.io/illustrated-bert/">Illustrated BERT by Jay Allammar</a>)

In [ ]:
Image(filename='images/gpt.png')

<a id="fig2">Figure 2.</a> Decoder only GPT model with only forward look direction (source: <a href="http://jalammar.github.io/illustrated-bert/">Illustrated BERT by Jay Allammar</a>)

In [ ]:
Image(filename='images/elmo-gpt-bert.png')


<a id="fig3">Figure 3.</a> A comparison of ELMo, GPT, and BERT (source: Dive into Deep Learning by Zhang et al., (2021) [[7]](#ref7))

<h3>Input Represenation</h3>

When handling input data BERT requires some special formatting for its inputs. If we're dealing with one sentence or a piece of text, we start with a special token called `<cls>`, followed by the tokens (words or pieces of words) from our sentence, and we end with another special token called `<sep>`. This whole sequence is our BERT input.

If we're dealing with two sentences, we start again with `<cls>`, then the tokens from the first sentence, then `<sep>`, the tokens from the second sentence, and another `<sep>` at the end.

These `<cls>` and `<sep>` tokens help BERT understand where sentences start, end, and separate [Figure 4](#fig4).

**'[CLS]` token**

To differentiate between two sentences in one input, BERT uses something called segment embeddings. This is just a way to give BERT a little extra information about which sentence a word came from. For single sentence inputs, it just uses one type of segment embedding.

The function you mentioned, get_tokens_and_segments, takes in either one or two sentences, and gives back the tokens for BERT's input and the segment IDs that go along with them.

The first input token is supplied with a special `[CLS]` token. Each position outputs a vector of size `hidden_size` (768 in BERT Base) where the aim is to focus on the output of only the first position - aka `[CLS]` [Figure 5](#fig5). 

In the context of the BERT model, the CLS token stands for <b>"classification token"</b>. It is a special token that is added to the start of each input sequence. The final hidden state (output vector) corresponding to this token is used as the aggregate sequence representation for classification tasks. The purpose of this CLS token is to effectively summarize the entire sequence (sentence or paragraph) into a single vector that can be used for classification tasks.

let's consider a simpler example:

Text: "The cat sat on the mat."

If we're using BERT to analyze this sentence, we first insert the CLS token: "[CLS] The cat sat on the mat."

As BERT reads the sentence, it keeps updating the CLS token. It's like a student underlining key ideas in a textbook. By the time BERT reads "mat", the CLS token could encapsulate the main action: "The cat is sitting on something."

So, if we ask BERT, "What is the cat doing?" It can refer to the CLS token, its 'highlighted notes', and answer, "The cat is sitting."

In simple terms, you can think of the CLS token as a symbol that tells the BERT model, "Okay, here starts a new piece of text for you to analyze." The model then uses the information processed for this token to perform tasks that require understanding the entire piece of text as a whole, such as sentiment analysis.

In [ ]:
Image(filename="images/bert-input.png")

<a id="fig4">Figure 4.</a> The embeddings of the BERT input sequence are the sum of the token embeddings, segment embeddings, and positional embeddings. (source: Dive into Deep Learning by Zhang et al., (2021) [[7]](#ref7))

In [ ]:
Image(filename='images/bert-output-vector.png')

**`get_tokens_and_segments` function**

This is a function that prepares inputs for BERT.

The function `get_tokens_and_segments` takes two arguments: `tokens_a` and `tokens_b`. These are lists of tokens (words or subwords) for the first and the second sentence, respectively. The second sentence is optional, so `tokens_b` can be None.

1. `tokens = ['<cls>'] + tokens_a + ['<sep>']`: This line creates a new list called `tokens` that starts with the special beginning-of-sentence token `<cls>`, followed by all the tokens from the first sentence `tokens_a`, and then the special end-of-sentence token `<sep>`.

2. `segments = [0] * (len(tokens_a) + 2)`: This line creates a list of segment IDs for the tokens. Each token from the first sentence gets a segment ID of 0. The `+ 2` is there to include the `<cls>` and `<sep>` tokens at the beginning and the end of the sequence.

3. The `if` statement checks if a second sentence is present (`tokens_b is not None`).

    - If there is a second sentence, the function adds the tokens from `tokens_b` to the `tokens` list, followed by another `<sep>` token.
  
    - It also adds segment IDs of 1 for each token in `tokens_b` (indicating they're part of the second sentence) to the `segments` list, and includes an extra 1 for the trailing `<sep>` token.

Finally, the function returns the `tokens` and `segments` lists. These are the prepared inputs for BERT, where `tokens` are the input words and `segments` are the identifiers indicating which sentence a word belongs to.


In [ ]:
def get_tokens_and_segments(tokens_a, tokens_b=None):
    """Get tokens of the BERT input sequence and their segment IDs."""
    tokens = ['<cls>'] + tokens_a + ['<sep>']  
    # 0 and 1 are marking segment A and B, respectively
    segments = [0] * (len(tokens_a) + 2)
    if tokens_b is not None:
        tokens += tokens_b + ['<sep>']
        segments += [1] * (len(tokens_b) + 1)
    return tokens, segments

<a id="#fig5">Figure 5.</a> BERT base model output vector (source: <a href="http://jalammar.github.io/illustrated-bert/">Illustrated BERT by Jay Allammar</a>)

1. <b>Masked Language Model (MLM)</b>

BERT alleviates the  unidirectionality constraint by using a “masked language model” (MLM) pre-training objective, inspired by the Cloze task (Taylor, 1953) [[6]](#ref6). The masked language model is a method where some words in a sentence are randomly hidden or "masked". The goal is to guess the original word that was hidden, using only the surrounding context as a clue. Unlike a method that only looks at words from left to right, this approach of masking and predicting allows the model to consider both the preceding and following context. This means it can train a more powerful tool that looks in both directions of text. 

During BERT's training, a random 15% of the words in a sentence are masked or hidden, and BERT's task is to accurately fill in the blanks. In essence, this is how BERT gets a direct lesson in understanding the English language and our word choices. Quite clever, don't you think?

For example, in the sentence "She went to the [MASK] to buy groceries", if the word "store" is masked, BERT's job would be to correctly predict that the missing word is "store".

In [ ]:
#@save
class MaskLM(nn.Module):
    """The masked language model task of BERT."""
    def __init__(self, vocab_size, num_hiddens, **kwargs):
        super(MaskLM, self).__init__(**kwargs)
        self.mlp = nn.Sequential(nn.LazyLinear(num_hiddens),
                                 nn.ReLU(),
                                 nn.LayerNorm(num_hiddens),
                                 nn.LazyLinear(vocab_size))

    def forward(self, X, pred_positions):
        num_pred_positions = pred_positions.shape[1]
        pred_positions = pred_positions.reshape(-1)
        batch_size = X.shape[0]
        batch_idx = torch.arange(0, batch_size)
        # Suppose that `batch_size` = 2, `num_pred_positions` = 3, then
        # `batch_idx` is `torch.tensor([0, 0, 0, 1, 1, 1])`
        batch_idx = torch.repeat_interleave(batch_idx, num_pred_positions)
        masked_X = X[batch_idx, pred_positions]
        masked_X = masked_X.reshape((batch_size, num_pred_positions, -1))
        mlm_Y_hat = self.mlp(masked_X)
        return mlm_Y_hat


2. <b>Next Sentence Prediction (NSP)</b>

Along with this MLM method, a "next sentence prediction" task is also used. This helps the model to understand and create links between two pieces of text. 
Essentially, BERT is tasked with predicting whether a particular sentence logically follows a preceding one or not.

For instance:

Sentence 1: "The dog loves to play fetch." <br>
Sentence 2: "He always returns the ball." (This would be identified as a logically connected sentence pair)

Sentence 1: "The sun is shining brightly today."<br>
Sentence 2: "Blue cheese tastes quite strong." (This would be identified as an illogical sentence pair)

During training, BERT is presented with an even mix of logically connected sentence pairs (50%) and random sentence pairs (50%). This helps BERT improve its accuracy in predicting the logical sequence of sentences.

Additionally it is worth noting that, BERT is trained on both MLM (50%) and NSP (50%) at the same time.



<h3> BASE and LARGE models</h3>

The paper [[2]](#ref2) presents two model sizes for BERT:

BERT BASE – Comparable in size to the OpenAI Transformer in order to compare performance
BERT LARGE – A ridiculously huge model which achieved the state of the art results reported in the paper

BERT is basically a trained Transformer Encoder stack.  This is a good time to direct you to <a href="https://github.com/elnaramammadova/Attention_is_all_you_need">Attention is all you need</a> which explains the Transformer model – a foundational concept for BERT and the concepts we’ll discuss next.

Both BERT architectures have a considerable number of encoder layers, referred to as Transformer Blocks in the original paper [[2]](#ref2). The Base version comprises 12 encoder layers, whereas the Large version includes 24 four [Figure 6](#fig6). These models also employ more substantial feedforward networks (with 768 hidden units for the Base model and 1024 for the Large model), and a greater number of attention heads (12 and 16, respectively) compared to the original Transformer reference implementation, which included 6 encoder layers, 512 hidden units, and 8 attention heads [[1]](#ref1).

Similar to the standard Transformer's encoder, BERT begins by accepting a word sequence as input, which proceeds up the hierarchical structure. Each layer performs self-attention, processes the results through a feed-forward network, and passes the output to the next encoder. Until this point, the architecture is virtually identical to the vanilla Transformer model, barring some size variations, which are configurable. The divergence from the traditional Transformer design becomes evident only when looking at the output stage.

In [ ]:
Image(filename='images/bert-base-bert-large-encoders.png')

<a id="fig6">Figure 6.</a> BERT BASE and LARGE Transformer Encoder Stack (source: <a href="http://jalammar.github.io/illustrated-bert/">Illustrated BERT by Jay Allammar</a>)


<h3>BERT Feature Extraction</h3>

BERT isn't only useful through fine-tuning; it can also create contextual word embeddings like ELMo. You can feed these embeddings into your existing model, a method that has been shown to give competitive results.

This process does not involve any fine-tuning of the BERT model; rather, it simply uses BERT to transform raw text data into a more useful, numerical format that other models can work with.

Here is a simple step-by-step breakdown of the process:

Tokenization: BERT uses a technique called WordPiece tokenization, breaking words into subwords if the word is not in its dictionary.

Adding special tokens: BERT requires specific formatting of its inputs which includes adding special tokens to mark the beginning ([CLS]) and separation/end of sentences ([SEP]).

Segment IDs: BERT can also take sentence pairs as inputs for tasks (like question answering). BERT uses segment IDs to differentiate between the two sentences. Sentence A is encoded as 0 and Sentence B as 1.

Embeddings: Once the sentences are properly formatted, they are passed through the BERT model. Since the model is a multi-layer bidirectional Transformer encoder, depending on the layer from which you extract embeddings, you may get different results. Researchers often use the second-to-last layer as it provides a good balance between the model's ability to generalize and its attention to specific words.

Extracting Features: The output of BERT is a vector (embedding) for each input token. These embeddings are contextually dependent, meaning the embedding for a particular word changes based on the sentence it is in. The original paper [[2]](#ref2) examines several choices: Last Hidden Layer, Sum of all 12 Layers, Second-to-last Hidden Layer, Sum of Last Four Hidden Layers and lastly Concat of the Last Four Hidden Layers, which achieved the highest score score of 96. [Figure 7](#fig7). 

In [ ]:
Image(filename='images/BERT-features.png')

<a id="#fig7">Figure 7.</a> BERT feature extraction by concatanating the last four hidden layer outputs (source: <a href="http://jalammar.github.io/illustrated-bert/">Illustrated BERT by Jay Allammar</a>). This vector works best as a contextualized embedding as described in the original BERT paper (with Dev F1 Score 96.1)

Unlike other large learning models like GPT-3, BERT’s source code is publicly accessible (view BERT’s code on <a href="https://github.com/google-research/bert">Github</a>) allowing BERT to be more widely used all around the world. This is a game-changer!

The `class BertModel` is pretty much identical to a vanilla Transformer encoder. If you are unfamiliar with the vanilla Transformer encoder architecture, feel free to review the in-depth and illustrated Pytorch implementation of the paper for detailed study <a href="https://github.com/elnaramammadova/Attention_is_all_you_need/tree/master">here</a>. 

BERT doesn’t look at words as tokens. Rather, it looks at WordPieces. `tokenization.py` is the tokenizer that would turns your words into wordPieces appropriate for BERT.

In simple terms, WordPieces is a way of breaking up a word into smaller chunks that BERT can understand better.

Think of it like this: imagine you come across a very long, complicated word that you've never seen before. It might be hard to understand the whole word at once. But if you break the word down into smaller parts that you recognize, it becomes much easier to get a rough idea of what the word might mean. That's essentially what BERT is doing with WordPieces.

For instance, consider the word 'unhappiness'. A WordPieces tokenizer might break it down into 'un', '##happy', and '##ness'. These smaller pieces are easier for BERT to manage. If BERT doesn't recognize a word, it tries to break it down further until it finds pieces it recognizes.

So when we talk about BERT using WordPieces instead of whole words, it means that BERT is working with these smaller chunks of words, rather than trying to understand each word as a single unit. This helps BERT handle a wider range of language and vocabulary, including words it hasn't seen before.


<h2>Fine-tune BERT model</h2>
(source: <a href="https://huggingface.co/docs/transformers/training#finetune-a-pretrained-model">Fine-tune a Pre-Trained Model</a>)


<h3>Dataset</h3>

Loading the <a href="https://huggingface.co/datasets/yelp_review_full">Yelp Reviews<a> dataset.

In [ ]:
dataset = load_dataset("yelp_review_full")

In [ ]:
dataset["train"][100]

<h4>Tokenize Dataset</h4>

- Utilize a tokenizer to organize your text data.
- Implement a padding strategy to fill in extra spaces in the text.
- Apply a truncation strategy to cut down lengthy text sequences.
- Use the 'map method' from 🤗 Datasets to:
- Efficiently preprocess your entire dataset at once.
- Apply the above points in one go.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased") # Load tokenizer

'''
The function tokenize_function takes in a set of examples, each containing some text. It then applies the tokenizer to this text.

examples["text"]: This accesses the "text" field in the provided examples.
padding="max_length": This argument ensures that all tokenized sequences are of the same length. If a sequence is shorter than the maximum length, it gets filled (padded) with extra tokens until it reaches the maximum length.
truncation=True: If a tokenized sequence exceeds the maximum length, this argument ensures the sequence is cut (truncated) down to the maximum length.
'''
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True) 

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Since we are only experimenting, let's get smaller subset of the dataset in order to save on runtime. 

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

<h3>Train</h3>

We can train using with Pytorch Trainer or native Pytorch. We will do both. 

<h4>PyTorch Trainer</h4>

The Trainer class within Transformers simplifies the training process by offering an optimized environment for these specific models. It supports an array of features including logging, gradient accumulation, and mixed precision, eliminating the need for a manually written training loop.

- load our model
- specify expected labels = 5 (as per Yelp Review dataset <a href="https://huggingface.co/datasets/yelp_review_full#data-fields">card<a>)

In [ ]:
# <https://huggingface.co/transformers/v3.0.2/model_doc/auto.html#automodelforsequenceclassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

# The pretrained head of the BERT model is discarded, and replaced with a randomly initialized 
# classification head. You will fine-tune this new model head on your sequence classification task, 
# transferring the knowledge of the pretrained model to it.

In [ ]:
from transformers import TrainingArguments

<h2>REFERENCES</h2>

<a id="ref1">[1]</a> Vaswani, A.; Shazeer, N.; Parmar, N.; Uszkoreit, J.; Jones, L.; Gomez, A. N.; Kaiser, Ł. & Polosukhin, I. (2017), Attention is all you need, in 'Advances in Neural Information Processing Systems' , pp. 5998--6008 . [online] Available at: https://arxiv.org/abs/1706.03762 <br>
<a id="ref2">[2]</a> Devlin, J., Chang, M.W., Lee, K. and Toutanova, K., 2018. BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding. [online] Available at: https://arxiv.org/abs/1810.04805 <br>
<a id="ref3">[3]</a> Turian, J., Ratinov, L-A. and Bengio, Y., 2010. 'Word Representations: A Simple and General Method for Semi-Supervised Learning'. Proceedings of the 48th Annual Meeting of the Association for Computational Linguistics, pp.384-394 . [online] Available at: https://aclanthology.org/P10-1040.pdf <br>
<a id="ref4">[4]</a> Mikolov, T., Chen, K., Corrado, G., & Dean, J. (2013). Efficient estimation of word representations in vector space. arXiv preprint arXiv:1301.3781. Available at: https://arxiv.org/pdf/1301.3781.pdf <br>
<a id="ref5">[5]</a> Pennington, J., Socher, R., & Manning, C. (2014). GloVe: Global Vectors for Word Representation. In Proceedings of the 2014 Conference on Empirical Methods in Natural Language Processing (EMNLP), pp. 1532–1543. Available at: https://nlp.stanford.edu/pubs/glove.pdf
<a id="ref6">[6]</a> Taylor, I. (1953). 'Cloze procedure: a new tool for measuring readability'. Journalism Bulletin, 30(4), pp. 415-433. Available at: https://gwern.net/doc/psychology/writing/1953-taylor.pdf <br>
<a id="ref7">[7]</a> Zhang, A., Lipton, Z.C., Li, M. and Smola, A.J., 2021. Dive into Deep Learning. [online] Available at: https://doi.org/10.48550/arXiv.2106.11342 <br>